In [32]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
import numpy as np
import spacy 
from transformers import AutoModel, AutoTokenizer
import torch

In [33]:
df_co=pd.read_excel("course_outcomes_combined_single_sheet.xlsx")

In [34]:
df_co

,No,Course Outcomes,Verbs,Assessments
0,CO1,Understand the essence of research and the\rim...,NaN,NaN
1,CO2,Explore the fundamental concepts of data science,NaN,NaN
2,CO3,Understand various machine learning algorithms...,NaN,NaN
3,CO4,Learn to think through the ethics surrounding ...,NaN,NaN
4,CO5,Create scientific reports according to specifi...,NaN,NaN
...,...,...,...,...
128,CO4,Designcomputationalexperimentsfortrainingandev...,NaN,NaN
129,CO1,Understand the fundamental principles of image...,NaN,NaN
130,CO2,Develop proficiency in image enhancement and s...,NaN,NaN
131,CO3,Develop skills in object detection and recogni...,NaN,NaN


In [35]:
df_co['Course Outcomes'].iloc[0]

'Understand the essence of research and the\rimportance of research methods and methodology'

In [36]:
df_co

,No,Course Outcomes,Verbs,Assessments
0,CO1,Understand the essence of research and the\rim...,NaN,NaN
1,CO2,Explore the fundamental concepts of data science,NaN,NaN
2,CO3,Understand various machine learning algorithms...,NaN,NaN
3,CO4,Learn to think through the ethics surrounding ...,NaN,NaN
4,CO5,Create scientific reports according to specifi...,NaN,NaN
...,...,...,...,...
128,CO4,Designcomputationalexperimentsfortrainingandev...,NaN,NaN
129,CO1,Understand the fundamental principles of image...,NaN,NaN
130,CO2,Develop proficiency in image enhancement and s...,NaN,NaN
131,CO3,Develop skills in object detection and recogni...,NaN,NaN


## Indentify the Verbs from the Course Outcome

In [37]:
course_outcomes=[]
for i in range(len(df_co)):
    data=df_co['Course Outcomes'].iloc[i]
    course_outcomes.append(data)

In [38]:
course_outcomes

['Understand the essence of research and the\rimportance of research methods and methodology',
 'Explore the fundamental concepts of data science',
 'Understand various machine learning algorithms used in data science process',
 'Learn to think through the ethics surrounding privacy, data sharing and algorithmi\rdecision- making',
 'Create scientific reports according to specified standards',
 'able to understand the concept of the random variable and expectation for discrete\rand continuous data',
 'evaluate condition probabilities and conditional expectations',
 'gain the knowledge of applications of discrete distributions in Data Science',
 'identify the applications of continuous distributions in Data Science',
 'apply Chebychevs inequality to verify the convergence of sequence in probability',
 'Understand the fundamentals of programming languages.',
 'Understand the design paradigms of programming languages.',
 'To examine expressions, subprograms and their parameters.',
 'Demons

In [39]:
verbs=['VB','VBP','VBD','VBG','VBN']

In [40]:
course_verbs=[]
for i in range(0,len(course_outcomes)):
    review=course_outcomes[i]
    review=review.lower()
    review=review.split()
    review=nltk.pos_tag(review)
    review=[word for word,tag in review if tag in verbs]
    course_verbs.append(review)

In [41]:
course_verbs

[['understand'],
 [],
 ['learning', 'used'],
 ['think', 'surrounding'],
 ['according', 'specified'],
 ['understand'],
 [],
 ['gain'],
 ['identify'],
 ['verify'],
 ['understand', 'programming'],
 ['understand', 'programming'],
 ['examine'],
 ['visualize'],
 ['apply'],
 [],
 ['demonstrate'],
 ['understand'],
 ['programming', 'solve', 'given'],
 ['demonstrate'],
 [],
 [],
 ['demonstrate', 'using'],
 ['given', 'solve', 'using'],
 ['infer'],
 ['analyze'],
 [],
 ['designing', 'including'],
 ['understand', 'deciding', 'know', 'do', 'admit'],
 [],
 ['understanding'],
 ['develop', 'using'],
 ['demonstrate'],
 ['apply', 'sampling', 'testing'],
 ['estimate', 'using', 'using'],
 ['using', 'using'],
 ['create'],
 ['describethemaintechnologiesandmethodscurrentlyusedincreating'],
 ['inherent'],
 ['using'],
 ['develop', 'writing'],
 ['programming', 'solve'],
 ['develop'],
 ['understand'],
 ['understand', 'are', 'formulated', 'solved'],
 ['learning', 'solve'],
 ['formulate'],
 ['identify'],
 ['identify

## Load Blooms Taxonomy Pdf

In [42]:
df_blooms=pd.read_excel("Blooms_index.xlsx")

In [43]:
df_blooms

,Level:,Verb,Examples of Appropriate Assessments
0,Remembering: can the student recall or remembe...,Recall Recognize Identify,Objective test items such as fill-in-the-blank...
1,Understanding: can the student explain ideas o...,Interpret Exemplify Classify Summarize Infer C...,"Activities such as papers, exams, problem sets..."
2,Applying: can the student use the information ...,Apply Execute Implement,"Activities such as problem sets, performances,..."
3,Analyzing: can the student distinguish between...,Analyze Differentiate Organize Attribute,"Activities such as case studies, critiques, la..."
4,Evaluating: can the student justify a stand or...,Evaluate Check Critique Assess,"Activities such as journals, diaries, critique..."
5,Creating: can the student create new product o...,Create Generate Plan Produce Design,"Activities such as research projects, musical ..."


In [44]:
df_blooms['Verb '] = df_blooms['Verb '].str.split()

In [45]:
df_blooms['Verb ']

0                        [Recall, Recognize, Identify]
1    [Interpret, Exemplify, Classify, Summarize, In...
2                          [Apply, Execute, Implement]
3        [Analyze, Differentiate, Organize, Attribute]
4                  [Evaluate, Check, Critique, Assess]
5            [Create, Generate, Plan, Produce, Design]
Name: Verb , dtype: object

In [46]:
flattened_course_verbs = [verb[0] for verb in course_verbs if verb]

In [47]:
flattened_course_verbs

['understand',
 'learning',
 'think',
 'according',
 'understand',
 'gain',
 'identify',
 'verify',
 'understand',
 'understand',
 'examine',
 'visualize',
 'apply',
 'demonstrate',
 'understand',
 'programming',
 'demonstrate',
 'demonstrate',
 'given',
 'infer',
 'analyze',
 'designing',
 'understand',
 'understanding',
 'develop',
 'demonstrate',
 'apply',
 'estimate',
 'using',
 'create',
 'describethemaintechnologiesandmethodscurrentlyusedincreating',
 'inherent',
 'using',
 'develop',
 'programming',
 'develop',
 'understand',
 'understand',
 'learning',
 'formulate',
 'identify',
 'identify',
 'use',
 'describe',
 'apply',
 'using',
 'using',
 'understand',
 'apply',
 'identify',
 'present',
 'using',
 'analyse',
 'differentiate',
 'forecast',
 'understand',
 'understand',
 '(cnn)',
 'evaluatethechallengesintraining',
 'understand',
 'identify',
 'understand',
 'illustrate',
 'illustrate',
 'utilizing',
 'describe',
 'analyze',
 'based',
 'demonstrate',
 'be',
 'apply',
 'knowle

In [48]:
flattened_df_verbs = [' '.join(verbs) for verbs in df_blooms['Verb ']]

In [49]:
flattened_df_verbs

['Recall Recognize Identify',
 'Interpret Exemplify Classify Summarize Infer Compare Explain',
 'Apply Execute Implement',
 'Analyze Differentiate Organize Attribute',
 'Evaluate Check Critique Assess',
 'Create Generate Plan Produce Design']

In [50]:
blooms_verbs=[]
for i in range(len(flattened_df_verbs)):
    review=flattened_df_verbs[i]
    review=review.split()
    blooms_verbs.append(review)

In [51]:
blooms_verbs

[['Recall', 'Recognize', 'Identify'],
 ['Interpret',
  'Exemplify',
  'Classify',
  'Summarize',
  'Infer',
  'Compare',
  'Explain'],
 ['Apply', 'Execute', 'Implement'],
 ['Analyze', 'Differentiate', 'Organize', 'Attribute'],
 ['Evaluate', 'Check', 'Critique', 'Assess'],
 ['Create', 'Generate', 'Plan', 'Produce', 'Design']]

In [52]:
# Load SpaCy model
nlp = spacy.load("en_core_web_md")

# Function to get the SpaCy vector for a word
def get_word_vector(word):
    doc = nlp(word)
    if doc.has_vector:  # Check if the word has a vector
        return doc.vector
    else:
        return np.zeros(nlp.vocab.vectors_length)  # Return a zero vector if word not in vocabulary

# Convert course verbs to vectors
course_verb_vectors = np.array([get_word_vector(verb) for verb in flattened_course_verbs])

# Set similarity threshold
threshold = 0.5  

# Dictionary to store matching assessments
matching_assessments = {}

for i, course_verb in enumerate(flattened_course_verbs):
    matching_assessments[course_verb] = []
    
    # Compare with each Bloom's category
    for cat_index, blooms_category in enumerate(blooms_verbs):
        blooms_verb_vectors = np.array([get_word_vector(verb) for verb in blooms_category])
        
        # Calculate cosine similarity between the course verb and each verb in the Bloom's category
        sim_scores = cosine_similarity([course_verb_vectors[i]], blooms_verb_vectors).flatten()
        
        # Check if any similarity score in this category is above the threshold
        if any(score > threshold for score in sim_scores):
            matching_assessments[course_verb].append(df_blooms.iloc[cat_index]['Examples of Appropriate Assessments'])

# Print results
for verb, assessments in matching_assessments.items():
    print(f"Verbs: {verb}")
    print(f"Assessments for course verb '{verb}': {assessments}\n")



Verbs: understand
Assessments for course verb 'understand': ['Objective test items such as fill-in-the-blank, matching, labeling, or multiple-choice questions that require students to: recall or recognize terms, facts, and concepts •', 'Activities such as papers, exams, problem sets, class discussions, or concept maps that require students to: summarize readings, films, or speeches • compare and contrast two or more theories, events, or processes • classify or categorize cases, elements, or events using established criteria • paraphrase documents or speeches • find or identify examples or illustrations of a concept or principle •', 'Activities such as case studies, critiques, labs, papers, projects, debates, or concept maps that require students to: discriminate or select relevant and irrelevant parts • determine how elements function together • determine bias, values, or underlying intent in presented material •', 'Activities such as journals, diaries, critiques, problem sets, product

In [53]:

verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']  # POS tags for verbs

# Function to extract verbs from the course outcomes
def extract_verbs(course_outcomes):
    course_verbs = []
    for outcome in course_outcomes:
        words = outcome.split()
        tagged_words = nltk.pos_tag(words)
        verbs_in_outcome = [word for word, tag in tagged_words if tag in verbs]
        course_verbs.append(verbs_in_outcome)
    return course_verbs

# Function to get SpaCy word vectors for words
def get_word_vector(word):
    doc = nlp(word)
    if doc.has_vector:
        return doc.vector
    else:
        return np.zeros(nlp.vocab.vectors_length)  # Return zero vector if word not in vocab

# Convert course verbs to vectors (assuming course_verbs is a list of verb lists)
flattened_course_verbs = [verb for sublist in course_verbs for verb in sublist]
course_verb_vectors = np.array([get_word_vector(verb) for verb in flattened_course_verbs])

# Set a similarity threshold
threshold = 0.5

# Assuming 'blooms_verbs' is your list of Bloom's verbs categories (from your example)
matching_assessments = {}

# Iterate over course verbs and find matching Bloom's verbs & assessments
for i, course_verb in enumerate(flattened_course_verbs):
    matching_assessments[course_verb] = []
    
    # Compare with each Bloom's category
    for cat_index, blooms_category in enumerate(blooms_verbs):
        blooms_verb_vectors = np.array([get_word_vector(verb) for verb in blooms_category])
        
        # Calculate cosine similarity between the course verb and each Bloom's verb
        sim_scores = cosine_similarity([course_verb_vectors[i]], blooms_verb_vectors).flatten()
        
        # Check if any similarity score in this category is above the threshold
        if any(score > threshold for score in sim_scores):
            matching_assessments[course_verb].append(df_blooms.iloc[cat_index]['Examples of Appropriate Assessments'])

# Print results for assessment mapping
for verb, assessments in matching_assessments.items():
    print(f"Verbs: {verb}")
    print(f"Assessments for course verb '{verb}': {assessments}\n")

# Now, map the extracted verbs and assessments back to your df_co DataFrame
df_co['Verbs'] = [', '.join(verbs) for verbs in course_verbs]

# To fill 'Assessments' column based on matching assessments
df_co['Assessments'] = df_co['Verbs'].apply(lambda verbs: 
                                            ', '.join([assessment for verb in verbs.split(', ') 
                                                       if verb in matching_assessments and matching_assessments[verb] 
                                                       for assessment in matching_assessments[verb]]))


Verbs: understand
Assessments for course verb 'understand': ['Objective test items such as fill-in-the-blank, matching, labeling, or multiple-choice questions that require students to: recall or recognize terms, facts, and concepts •', 'Activities such as papers, exams, problem sets, class discussions, or concept maps that require students to: summarize readings, films, or speeches • compare and contrast two or more theories, events, or processes • classify or categorize cases, elements, or events using established criteria • paraphrase documents or speeches • find or identify examples or illustrations of a concept or principle •', 'Activities such as case studies, critiques, labs, papers, projects, debates, or concept maps that require students to: discriminate or select relevant and irrelevant parts • determine how elements function together • determine bias, values, or underlying intent in presented material •', 'Activities such as journals, diaries, critiques, problem sets, product

In [54]:
df_co

,No,Course Outcomes,Verbs,Assessments
0,CO1,Understand the essence of research and the\rim...,understand,Objective test items such as fill-in-the-blank...
1,CO2,Explore the fundamental concepts of data science,,
2,CO3,Understand various machine learning algorithms...,"learning, used",Objective test items such as fill-in-the-blank...
3,CO4,Learn to think through the ethics surrounding ...,"think, surrounding",Objective test items such as fill-in-the-blank...
4,CO5,Create scientific reports according to specifi...,"according, specified",Objective test items such as fill-in-the-blank...
...,...,...,...,...
128,CO4,Designcomputationalexperimentsfortrainingandev...,"learning, solving",Objective test items such as fill-in-the-blank...
129,CO1,Understand the fundamental principles of image...,understand,Objective test items such as fill-in-the-blank...
130,CO2,Develop proficiency in image enhancement and s...,develop,Objective test items such as fill-in-the-blank...
131,CO3,Develop skills in object detection and recogni...,develop,Objective test items such as fill-in-the-blank...


In [55]:
df_co.to_excel("Course Assessments.xlsx")

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
import numpy as np
import spacy 
from transformers import AutoModel, AutoTokenizer
import torch

df_co=pd.read_excel("course_outcomes_combined_single_sheet.xlsx")
course_outcomes=[]
for i in range(len(df_co)):
    data=df_co['Course Outcomes'].iloc[i]
    course_outcomes.append(data)

verbs=['VB','VBP','VBD','VBG','VBN']
course_verbs=[]
for i in range(0,len(course_outcomes)):
    review=course_outcomes[i]
    review=review.lower()
    review=review.split()
    review=nltk.pos_tag(review)
    review=[word for word,tag in review if tag in verbs]
    course_verbs.append(review)
df_blooms=pd.read_excel("Blooms_index.xlsx")
df_blooms['Verb '] = df_blooms['Verb '].str.split()
flattened_course_verbs = [verb[0] for verb in course_verbs if verb]
flattened_df_verbs = [' '.join(verbs) for verbs in df_blooms['Verb ']]
blooms_verbs=[]
for i in range(len(flattened_df_verbs)):
    review=flattened_df_verbs[i]
    review=review.split()
    blooms_verbs.append(review)

verbs = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']  # POS tags for verbs

# Function to extract verbs from the course outcomes
def extract_verbs(course_outcomes):
    course_verbs = []
    for outcome in course_outcomes:
        words = outcome.split()
        tagged_words = nltk.pos_tag(words)
        verbs_in_outcome = [word for word, tag in tagged_words if tag in verbs]
        course_verbs.append(verbs_in_outcome)
    return course_verbs

# Function to get SpaCy word vectors for words
def get_word_vector(word):
    doc = nlp(word)
    if doc.has_vector:
        return doc.vector
    else:
        return np.zeros(nlp.vocab.vectors_length)  # Return zero vector if word not in vocab

# Convert course verbs to vectors (assuming course_verbs is a list of verb lists)
flattened_course_verbs = [verb for sublist in course_verbs for verb in sublist]
course_verb_vectors = np.array([get_word_vector(verb) for verb in flattened_course_verbs])

# Set a similarity threshold
threshold = 0.5

# Assuming 'blooms_verbs' is your list of Bloom's verbs categories (from your example)
matching_assessments = {}

# Iterate over course verbs and find matching Bloom's verbs & assessments
for i, course_verb in enumerate(flattened_course_verbs):
    matching_assessments[course_verb] = []
    
    # Compare with each Bloom's category
    for cat_index, blooms_category in enumerate(blooms_verbs):
        blooms_verb_vectors = np.array([get_word_vector(verb) for verb in blooms_category])
        
        # Calculate cosine similarity between the course verb and each Bloom's verb
        sim_scores = cosine_similarity([course_verb_vectors[i]], blooms_verb_vectors).flatten()
        
        # Check if any similarity score in this category is above the threshold
        if any(score > threshold for score in sim_scores):
            matching_assessments[course_verb].append(df_blooms.iloc[cat_index]['Examples of Appropriate Assessments'])

# Print results for assessment mapping
for verb, assessments in matching_assessments.items():
    print(f"Verbs: {verb}")
    print(f"Assessments for course verb '{verb}': {assessments}\n")

# Now, map the extracted verbs and assessments back to your df_co DataFrame
df_co['Verbs'] = [', '.join(verbs) for verbs in course_verbs]

# To fill 'Assessments' column based on matching assessments
df_co['Assessments'] = df_co['Verbs'].apply(lambda verbs: 
                                            ', '.join([assessment for verb in verbs.split(', ') 
                                                       if verb in matching_assessments and matching_assessments[verb] 
                                                       for assessment in matching_assessments[verb]]))
df_co.to_excel("Course Assessments.xlsx")

KeyboardInterrupt: 